In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numba

import scipy.sparse as sparse
import scipy.io as io

import tskit as ts
import tstrait
import msprime

import sys
sys.path.append("../tsblup") # change the path to where you saved tsblup
from tsblup.operations import *
from tsblup.matrices import *
from tsblup.simulations import *

In [2]:
###### sim ancestry
ts = msprime.sim_ancestry(
    samples=1000,
    recombination_rate=1e-8,
    sequence_length=100_000,
    population_size=10_000,
    random_seed=100,
)

###### chunck edges to have unique descendant nodes
split_ts = split_upwards(ts)

###### assign effect size to positions
np.random.seed(101)
beta = np.random.normal(loc=0, scale=0.1, size=int(split_ts.sequence_length))

###### simulate trait
rate = 2e-8
trait_sim = GeneticValueSimulator(split_ts, beta)
trait_sim.sim_genetic_value(rate=rate, ignore_parent=True, random_seed=1, model=msprime.InfiniteAlleles())

##### edge based matrices
A_edge = edge_adjacency(split_ts).T # transpose to make A upper triangular
T_edge = sparse.identity(split_ts.num_edges) - A_edge
Z_edge = edge_individual_matrix(split_ts).T

##### node based matrices
A_node, _ = node_transmission_matrix(ts)
T_node = sparse.identity(ts.num_nodes) - A_node
Z_node = node_individual_matrix(ts).T

##### trait values
val_tstrait = trait_sim.genetic_df.genetic_value.values
val_trait = val_tstrait + np.random.normal(loc=0, scale=np.std(val_tstrait), size=val_tstrait.shape[0])

In [3]:
io.mmwrite('outputs/edge/T.edge.mtx', T_edge)
io.mmwrite('outputs/edge/Q.edge.mtx', T_edge.T @ T_edge)
io.mmwrite('outputs/edge/Z.edge.mtx', Z_edge)

io.mmwrite('outputs/node/Q.node.mtx', T_node)
io.mmwrite('outputs/node/Q.node.mtx', T_node.T @ T_node)
io.mmwrite('outputs/node/Z.node.mtx', Z_node)

np.savetxt('outputs/geneticvalue.txt', val_tstrait)
np.savetxt('outputs/phenovalue.txt', val_trait)